In [54]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [55]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [56]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [57]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

happy completed
neutral completed
sad completed


In [58]:
print(train)

                               image  label
0         images/train\happy\im0.png  happy
1         images/train\happy\im1.png  happy
2        images/train\happy\im10.png  happy
3       images/train\happy\im100.png  happy
4      images/train\happy\im1000.png  happy
...                              ...    ...
17005     images/train\sad\im995.png    sad
17006     images/train\sad\im996.png    sad
17007     images/train\sad\im997.png    sad
17008     images/train\sad\im998.png    sad
17009     images/train\sad\im999.png    sad

[17010 rows x 2 columns]


In [59]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

happy completed
neutral completed
sad completed


In [60]:
print(test)
print(test['image'])

                             image  label
0        images/test\happy\im0.png  happy
1        images/test\happy\im1.png  happy
2       images/test\happy\im10.png  happy
3      images/test\happy\im100.png  happy
4     images/test\happy\im1000.png  happy
...                            ...    ...
4249     images/test\sad\im995.png    sad
4250     images/test\sad\im996.png    sad
4251     images/test\sad\im997.png    sad
4252     images/test\sad\im998.png    sad
4253     images/test\sad\im999.png    sad

[4254 rows x 2 columns]
0          images/test\happy\im0.png
1          images/test\happy\im1.png
2         images/test\happy\im10.png
3        images/test\happy\im100.png
4       images/test\happy\im1000.png
                    ...             
4249       images/test\sad\im995.png
4250       images/test\sad\im996.png
4251       images/test\sad\im997.png
4252       images/test\sad\im998.png
4253       images/test\sad\im999.png
Name: image, Length: 4254, dtype: object


In [61]:
from tqdm.notebook import tqdm

In [62]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [63]:
train_features = extract_features(train['image']) 

  0%|          | 0/17010 [00:00<?, ?it/s]

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [64]:
test_features = extract_features(test['image'])

  0%|          | 0/4254 [00:00<?, ?it/s]

In [65]:
x_train = train_features/255.0
x_test = test_features/255.0

In [66]:
from sklearn.preprocessing import LabelEncoder

In [67]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [68]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [69]:
y_train = to_categorical(y_train,num_classes = 3)
y_test = to_categorical(y_test,num_classes = 3)

In [70]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(3, activation='softmax'))

In [71]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [72]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 5, validation_data = (x_test,y_test)) 

Epoch 1/5
133/133 [==============================] - 192s 1s/step - loss: 1.0816 - accuracy: 0.4218 - val_loss: 1.0812 - val_accuracy: 0.4170
Epoch 2/5
133/133 [==============================] - 154s 1s/step - loss: 1.0478 - accuracy: 0.4487 - val_loss: 0.9088 - val_accuracy: 0.5470
Epoch 3/5
133/133 [==============================] - 151s 1s/step - loss: 0.8880 - accuracy: 0.5713 - val_loss: 0.8056 - val_accuracy: 0.6218
Epoch 4/5
133/133 [==============================] - 151s 1s/step - loss: 0.8129 - accuracy: 0.6195 - val_loss: 0.7449 - val_accuracy: 0.6594
Epoch 5/5
133/133 [==============================] - 151s 1s/step - loss: 0.7710 - accuracy: 0.6440 - val_loss: 0.7086 - val_accuracy: 0.6775


In [73]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [74]:
from keras.models import model_from_json

In [75]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [76]:
label = ['happy','neutral','sad']

In [77]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [78]:
image = 'images/train/happy/im1.png'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of happy
1/1 [==============================] - 0s 114ms/step
model prediction is  happy


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [79]:
import matplotlib.pyplot as plt
%matplotlib inline